# Input several news articles into the Stardog DNA repositories

There are 8 articles from different sources on the topics of the UN Gaza Ceasefire vote, the lowering of the Trump bond payment, the start of the Trump NY trial in mid-April, and the FL Supreme Court upholding the 6-week abortion ban.

Make sure that: 
* The DNA flask is running (execute 'flask run' from the dna directory)

In [1]:
import json
import requests

In [2]:
# Delete repositories (if they exist)
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=fl_abortion')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_trial')
print(response.status_code, response) 
response = requests.delete('http://127.0.0.1:5000/dna/v1/repositories?repository=landslide')
print(response.status_code, response) 

200 <Response [200]>
200 <Response [200]>
404 <Response [404]>


In [3]:
# Create the repositories for the news articles
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=landslide')
print(response.status_code, response) 
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=fl_abortion')
print(response.status_code, response) 
response = requests.post('http://127.0.0.1:5000/dna/v1/repositories?repository=trump_trial')
print(response.status_code, response) 
# Verify that the repositories were created
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories')
print(response.status_code, response.json()) 

201 <Response [201]>
201 <Response [201]>
201 <Response [201]>
200 [{'created': '2024-05-27T10:43:32', 'repository': 'fl_abortion'}, {'created': '2024-05-27T10:43:33', 'repository': 'trump_trial'}, {'created': '2024-05-27T10:43:31', 'repository': 'landslide'}]


In [4]:
# Articles for ingest
landslide_aljazeera = {
            "title": "UN estimates more than 670 killed in Papua New Guinea landslide",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/5/26/un-estimates-more-than-670-killed-in-papua-new-guinea-landslide"}
landslide_breitbart = {
            "title": "UN raises Papua New Guinea landslide death toll estimate to 670",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.breitbart.com/news/un-raises-papua-new-guinea-landslide-death-toll-estimate-to-670/"}
landslide_cnn = {
            "title": "More than 670 people feared dead after Papua New Guinea landslide, UN says",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.cnn.com/2024/05/26/world/papua-new-guinea-landslide-death-toll-intl/index.html"}
landslide_fox = {
            "title": "Hundreds reportedly dead after severe landslide strikes Papua New Guinea village",
            "published": "2024-05-24T00:00:00",
            "url": "https://www.foxnews.com/world/hundreds-feared-dead-severe-landslide-strikes-papua-new-guinea-village"}
landslide_huffpost = {
            "title": "U.N. Migration Agency Estimates More Than 670 Killed In Papua New Guinea Landslide",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.huffpost.com/entry/bc-as-papua-new-guinea-landslide_n_665327a8e4b09d0995d6dd6f"}
landslide_nyt = {
            "title": "Death Toll in Papua New Guinea Landslide Estimated to Be at Least 670",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.nytimes.com/2024/05/26/world/asia/papua-new-guinea-landslide-death.html"}
landslide_watimes = {
            "title": "U.N. estimates over 670 people died in a massive Papua New Guinea landslide as survivors seek safety",
            "published": "2024-05-26T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/may/26/un-estimates-over-670-people-died-in-massive-papua/"}
landslide_wsj = dict()


fl_abortion_aljazeera = {
            "title": "Biden denounces Florida abortion ruling as ‘outrageous’ as state vote looms",
            "published": "2024-04-02T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms"}
fl_abortion_breitbart = {
            "title": "Florida Supreme Court Upholds 15-Week Abortion Limit, But Voters Will Decide in November",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/04/01/florida-supreme-court-upholds-15-week-abortion-limit-but-voters-will-decide-in-november/"}
fl_abortion_cnn = {
            "title": "Florida six-week abortion ban will soon become law, but voters will consider a constitutional amendment this fall",
            "published": "2024-04-02T00:00:00",
            "url": "https://www.cnn.com/2024/04/01/politics/florida-six-week-abortion-ban/index.html"}
fl_abortion_fox = {
            "title": "Florida Supreme Court approves abortion ballot initiative while upholding 15-week ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.foxnews.com/politics/florida-supreme-court-approves-abortion-ballot-initiative-upholding-15-week-ban"}
fl_abortion_huffpost = {
            "title": "Top Florida Court Rules Against Protections For Abortion, Greenlights 6-Week Ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.huffpost.com/entry/florida-court-abortion-6-week-ban_n_65088b87e4b0f04446230d7e"}
fl_abortion_nyt = {
            "title": "Florida Court Allows 6-Week Abortion Ban, but Voters Will Get to Weigh In",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.nytimes.com/2024/04/01/us/florida-abortion-law-supreme-court.html"}
fl_abortion_watimes = {
            "title": "Florida Supreme Court upholds state’s 15-week ban on most abortions, paving way for 6-week ban",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/apr/1/florida-supreme-court-upholds-states-15-week-ban-2/"}
fl_abortion_wsj = {
            "title": "Florida Abortion Rulings to Keep Issue Front and Center in November",
            "published": "2024-03-25T00:00:00",
            "url": "https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c"}


trump_trial_aljazeera = {
            "title": "Six takeaways from first day of Trump’s New York hush money criminal trial",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.aljazeera.com/news/2024/4/15/five-takeaways-from-first-day-of-trumps-historic-new-york-cirminal-trial"}
trump_trial_breitbart = {
            "title": "Unprecedented Trump Criminal Trial Begins with Jury Selection",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.breitbart.com/politics/2024/04/15/unprecedented-trump-criminal-trial-begins-with-jury-selection/"}
trump_trial_cnn = {
            "title": "Trump has an unusual request on trial Day 1",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.cnn.com/2024/04/15/politics/trump-trial-day-1-what-matters/index.html"}
trump_trial_fox = {
            "title": "Trump hush money trial begins in New York City with jury selection",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.foxnews.com/politics/trump-bragg-hush-money-trial-begins-new-york-city-jury-selection"}
trump_trial_huffpost = {
            "title": "Manhattan Court To Find Jurors To Hear First-Ever Criminal Case Against An Ex-President",
            "published": "2024-04-12T00:00:00",
            "url": "https://www.huffpost.com/entry/trump-bond-reduced-fraud-appeal_n_660198c0e4b059750143d44c"}
trump_trial_nyt = {
            "title": "A Historic Day",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.nytimes.com/2024/04/15/us/politics/trump-trial-manhattan-first-day.html"}
trump_trial_watimes = {
            "title": "Trump’s history-making hush-money trial begins with challenge of picking a jury",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.washingtontimes.com/news/2024/apr/15/trumps-hush-money-trial-begins-with-challenge-of-p/"}
trump_trial_wsj = {
            "title": "Donald Trump Faces Deep Blue Jury Pool as Hush-Money Case Begins",
            "published": "2024-04-15T00:00:00",
            "url": "https://www.wsj.com/us-news/law/trump-hush-money-trial-jury-2bc58769"}

# repos = ['ceasefire', 'fl_abortion', 'trump_bond', 'trump_trial']
repos = ['fl_abortion', 'landslide', 'trump_trial']
# 2 ~neutral sources (Al Jazeera, WSJ), 3 right/conservative sources(Breitbart, Fox, WA Times)
#    3 left/liberal sources (CNN, Huffington Post, NYT)
sources = {'aljazeera': 'Al Jazeera',    # ~ neutral
           'wsj': 'Wall Street Journal',
           'breitbart': 'Breitbart',     # conservative
           'fox': 'Fox News',
           'watimes': 'Washington Times',
           'cnn': 'CNN',                 # liberal
           'huffpost': 'Huffington Post',
           'nyt': 'New York Times'}

In [5]:
# Function to ingest the first 10 sentences of the 8 news articles regarding a specific topic 
# (ingested to the 'topic' repository)
def ingest_articles(repo: str): 
    for source in sources:
        repo_source = f'{repo}_{source}'
        print('repo_source ', repo_source)
        try:
            with open(f'articles/{repo_source}.txt', 'r') as news:
                news_text = news.read()  
        except:
            print(f'No article text for {repo_source}')
            continue
        metadata = globals()[repo_source]
        req_dict = metadata
        req_dict["source"] = sources[source]
        req_dict["text"] = news_text
        response = requests.post(
            f'http://127.0.0.1:5000/dna/v1/repositories/narratives?repository={repo}&sentences=10',
            json=req_dict)
        print(response)
        print(response.status_code, response.json())
        print()

In [6]:
ingest_articles('fl_abortion')

ingest_articles('landslide')

ingest_articles('trump_trial')

repo_source  fl_abortion_aljazeera
<Response [201]>
201 {'narrativeDetails': {'narrativeId': '22b19206', 'narrativeMetadata': {'published': '2024-04-02T00:00:00', 'source': 'Al Jazeera', 'title': 'Biden denounces Florida abortion ruling as ‘outrageous’ as state vote looms', 'url': 'https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms'}, 'numberIngested': 10, 'numberOfSentences': 35, 'numberOfTriples': '560', 'processed': '2024-05-27T10:52:35'}, 'repository': 'fl_abortion'}

repo_source  fl_abortion_wsj
<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'b00dd7f1', 'narrativeMetadata': {'published': '2024-03-25T00:00:00', 'source': 'Wall Street Journal', 'title': 'Florida Abortion Rulings to Keep Issue Front and Center in November', 'url': 'https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c'}, 'numberIngested': 10, 'numberOfSentences': 36, 'numberOfTriples

<Response [201]>
201 {'narrativeDetails': {'narrativeId': 'eec026d2', 'narrativeMetadata': {'published': '2024-04-15T00:00:00', 'source': 'Wall Street Journal', 'title': 'Donald Trump Faces Deep Blue Jury Pool as Hush-Money Case Begins', 'url': 'https://www.wsj.com/us-news/law/trump-hush-money-trial-jury-2bc58769'}, 'numberIngested': 10, 'numberOfSentences': 40, 'numberOfTriples': '529', 'processed': '2024-05-27T12:56:13'}, 'repository': 'trump_trial'}

repo_source  trump_trial_breitbart
<Response [201]>
201 {'narrativeDetails': {'narrativeId': '8a0d9e6b', 'narrativeMetadata': {'published': '2024-04-15T00:00:00', 'source': 'Breitbart', 'title': 'Unprecedented Trump Criminal Trial Begins with Jury Selection', 'url': 'https://www.breitbart.com/politics/2024/04/15/unprecedented-trump-criminal-trial-begins-with-jury-selection/'}, 'numberIngested': 10, 'numberOfSentences': 20, 'numberOfTriples': '423', 'processed': '2024-05-27T13:01:30'}, 'repository': 'trump_trial'}

repo_source  trump_tri

In [7]:
# Check what articles are in the fl_abortion repository 
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=fl_abortion')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "22b19206",
   "narrativeMetadata": {
    "published": "2024-04-02T00:00:00",
    "source": "Al Jazeera",
    "title": "Biden denounces Florida abortion ruling as \u2018outrageous\u2019 as state vote looms",
    "url": "https://www.aljazeera.com/news/2024/4/2/biden-denounces-florida-abortion-ruling-as-outrageous-as-state-vote-looms"
   },
   "numberIngested": "10",
   "numberOfSentences": "35",
   "numberOfTriples": "560",
   "processed": "2024-05-27T10:52:35"
  },
  {
   "narrativeId": "b00dd7f1",
   "narrativeMetadata": {
    "published": "2024-03-25T00:00:00",
    "source": "Wall Street Journal",
    "title": "Florida Abortion Rulings to Keep Issue Front and Center in November",
    "url": "https://www.wsj.com/politics/elections/florida-abortion-rulings-to-keep-issue-front-and-center-in-november-1168511c"
   },
   "numberIngested": "10",
   "numberOfSentences": "36",
   "numberOfTriples": "741",
   "processed": "2024-05-27T11:02:49"
  },


In [8]:
# In the trump_trial repository
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=trump_trial')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "36ece2e3",
   "narrativeMetadata": {
    "published": "2024-04-15T00:00:00",
    "source": "Al Jazeera",
    "title": "Six takeaways from first day of Trump\u2019s New York hush money criminal trial",
    "url": "https://www.aljazeera.com/news/2024/4/15/five-takeaways-from-first-day-of-trumps-historic-new-york-cirminal-trial"
   },
   "numberIngested": "10",
   "numberOfSentences": "66",
   "numberOfTriples": "459",
   "processed": "2024-05-27T10:46:30"
  },
  {
   "narrativeId": "42f9d216",
   "narrativeMetadata": {
    "published": "2024-04-15T00:00:00",
    "source": "Al Jazeera",
    "title": "Six takeaways from first day of Trump\u2019s New York hush money criminal trial",
    "url": "https://www.aljazeera.com/news/2024/4/15/five-takeaways-from-first-day-of-trumps-historic-new-york-cirminal-trial"
   },
   "numberIngested": "10",
   "numberOfSentences": "66",
   "numberOfTriples": "474",
   "processed": "2024-05-27T10:48:57"
  },
  {
 

In [9]:
# In the landslide repository
response = requests.get('http://127.0.0.1:5000/dna/v1/repositories/narratives?repository=landslide')
print(response.status_code, json.dumps(response.json(), indent=1))

200 {
 "narratives": [
  {
   "narrativeId": "046395bf",
   "narrativeMetadata": {
    "published": "2024-05-26T00:00:00",
    "source": "Al Jazeera",
    "title": "UN estimates more than 670 killed in Papua New Guinea landslide",
    "url": "https://www.aljazeera.com/news/2024/5/26/un-estimates-more-than-670-killed-in-papua-new-guinea-landslide"
   },
   "numberIngested": "10",
   "numberOfSentences": "19",
   "numberOfTriples": "386",
   "processed": "2024-05-27T10:42:17"
  },
  {
   "narrativeId": "e4a53609",
   "narrativeMetadata": {
    "published": "2024-05-26T00:00:00",
    "source": "Al Jazeera",
    "title": "UN estimates more than 670 killed in Papua New Guinea landslide",
    "url": "https://www.aljazeera.com/news/2024/5/26/un-estimates-more-than-670-killed-in-papua-new-guinea-landslide"
   },
   "numberIngested": "10",
   "numberOfSentences": "19",
   "numberOfTriples": "399",
   "processed": "2024-05-27T11:58:31"
  },
  {
   "narrativeId": "b2cf6688",
   "narrativeMetadata

In [12]:
# Dump of the article meta-data and triples
for repo in ('fl_abortion', 'landslide', 'trump_trial'):
    response = requests.get(f'http://127.0.0.1:5000/dna/v1/repositories/narratives?repository={repo}')
    narr_details = response.json()['narratives']
    narr_ids = []
    for detail in narr_details:
        narr_ids.append(detail['narrativeId'])
    for narr_id in narr_ids:
        response = requests.get(
            f'http://127.0.0.1:5000/dna/v1/repositories/narratives/graphs?repository={repo}&narrativeId={narr_id}')
        resp_json = response.json()
        with open(f'dumps/{repo}_{narr_id}_details.json', 'w') as detail_out:
            detail_out.write(json.dumps(dict(resp_json['narrativeDetails'])))
        with open(f'dumps/{repo}_{narr_id}.ttl', 'w') as triples_out:
            for triple in resp_json['triples']:
                triples_out.write(triple)
    